In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.activations import swish
from tensorflow.keras.callbacks import TensorBoard
import time
import matplotlib.pyplot as plt
from matplotlib import gridspec

2024-03-06 17:45:22.800537: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 17:45:22.800761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 17:45:22.962701: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
NAME = "cifakeCNN{}".format(time.strftime("%Y%m%d-%H%M%S"))

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
#   try:
#     tf.config.set_logical_device_configuration(
#         gpus[0],
#         [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

In [5]:
# tf.config.threading.set_intra_op_parallelism_threads(1)
# tf.config.threading.set_inter_op_parallelism_threads(1)

In [6]:
ds_train = image_dataset_from_directory(
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train',
    labels='inferred',
    label_mode='binary',
    image_size=[32,32],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
    seed=69,
    #validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 100000 files belonging to 2 classes.


In [7]:
ds_test = image_dataset_from_directory(
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test',
    labels='inferred',
    label_mode='binary',
    image_size=[32,32],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
    seed=69,
    #validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 20000 files belonging to 2 classes.


In [8]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE


In [9]:
ds_train = (
    ds_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


In [10]:
ds_valid = (
    ds_test
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [11]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=12,
    min_delta=0.001,
    restore_best_weights=True)

In [12]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=0.0008)

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint
timestamp = time.strftime("%Y%m%d-%H%M%S")
model_checkpoint = ModelCheckpoint(
    f'/kaggle/working/best/best_model_{timestamp}.keras', monitor='val_loss', save_best_only=True, mode='min'
)

In [14]:
def MBConv6(input_tensor, hp=None, out_channels=32, expansion=6, use_se=True, dropout_rate=0.0):
    # Inverted Residual Structure
    # Pointwise Convolution  1 with Expansion Factor  6
    x = layers.Conv2D(expansion * input_tensor.shape[-1], (1,  1), padding='same', use_bias=False)(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # Depthwise Separable Convolution with stride (1,  1)
    x = layers.DepthwiseConv2D(kernel_size=(3,  3), strides=(1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # Pointwise Convolution  2
    x = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Dropout
    if dropout_rate >  0.0:
        x = layers.Dropout(dropout_rate)(x)

    # Bottleneck Convolution  
    x = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Squeeze-and-Excitation
    if use_se:
        se_shape = (1,  1, out_channels)
        se = layers.GlobalAveragePooling2D()(x)
        se = layers.Reshape(se_shape)(se)
        se = layers.Conv2D(out_channels //  4, (1,  1), padding='same', use_bias=True)(se)
        se = layers.Activation(swish)(se)
        se = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=True)(se)
        se = layers.Activation('sigmoid')(se)
        x = layers.Multiply()([x, se])

    # Residual
    if input_tensor.shape[-1] == out_channels:
        shortcut = input_tensor
    else:
        shortcut = layers.Conv2D(out_channels, (1,  1), strides=(1,  1), padding='same', use_bias=False)(input_tensor)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])

    return x
input_tensor = layers.Input(shape=(32,  32,  16)) 
output_tensor = MBConv6(input_tensor)
model = keras.models.Model(inputs=input_tensor, outputs=output_tensor)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32,    │          0 │ -                 │
│ (InputLayer)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │      1,536 │ input_layer[0][0] │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32,    │        384 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d    │ (None, 32, 32,    │        864 │ activation[0][0]  │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        384 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32,    │      3,072 │ activation_1[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32,    │      1,024 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1, 32)  │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 1, 1, 8)   │        264 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 1, 1, 8)   │          0 │ conv2d_3[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 1, 1, 32)  │        288 │ activation_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 1, 1, 32)  │          0 │ conv2d_4[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │        512 │ input_layer[0][0

 Total params: 8,712 (34.03 KB)

 Trainable params: 8,136 (31.78 KB)

 Non-trainable params: 576 (2.25 KB)

In [15]:

def MBConv1(input_tensor, hp=None, expansion=4, out_channels=16, strides=(1,   1), use_se=True):
    # Expansion
    x = layers.Conv2D(expansion, (1,   1), padding='same', use_bias=False)(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # Depthwise 
    x = layers.DepthwiseConv2D(kernel_size=(3,   3), strides=strides, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x) 

    # Pointwise 
    x = layers.Conv2D(out_channels, (1,   1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Squeeze-and-Excitation
    if use_se:
        se_shape = (1,  1, out_channels)
        se = layers.GlobalAveragePooling2D()(x)
        se = layers.Reshape((1,  1, out_channels))(se) 
        se = layers.Conv2D(out_channels //  2, (1,  1), padding='same', use_bias=True)(se) 
        se = layers.Activation(swish)(se)   
        se = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=True)(se) 
        se = layers.Activation('sigmoid')(se)
        se = layers.Reshape(se_shape[1:])(se)  # Reshape to original
        x = layers.Multiply()([x, se])

    # Residual
    if input_tensor.shape[-1] == out_channels and strides == (1,   1):
        x = layers.Add()([x, input_tensor])
    else:
        input_tensor = layers.Conv2D(out_channels, (1,   1), strides=strides, padding='same', use_bias=False)(input_tensor)
        x = layers.Add()([x, input_tensor])

    return x

input_tensor = layers.Input(shape=(32,  32,  3)) 
output_tensor = MBConv1(input_tensor)
model = keras.models.Model(inputs=input_tensor, outputs=output_tensor)

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32, 4) │         12 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32, 4) │         16 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 32, 32, 4) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_1  │ (None, 32, 32, 4) │         36 │ activation_4[0][… │
│ (DepthwiseConv2D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32, 4) │         16 │ depthwise_conv2d… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 32, 32, 4) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 32, 32,    │         64 │ activation_5[0][… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_7[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 16)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 1, 1, 16)  │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 1, 1, 8)   │        136 │ reshape_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 1, 1, 8)   │          0 │ conv2d_8[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 1, 1, 16)  │        144 │ activation_6[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 1, 1, 16)  │          0 │ conv2d_9[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 1, 16)     │          0 │ activation_7[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Multiply)          │ 16)               │            │ reshape_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 32, 32,    │         48 │ input_layer_1[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 536 (2.09 KB)

 Trainable params: 488 (1.91 KB)

 Non-trainable params: 48 (192.00 B)

In [16]:
import keras_tuner
from keras_tuner import RandomSearch, Hyperband
from tensorflow.keras.models import Model

def build_model(hp=None):
    activation_function = 'leaky_relu' if hp.Boolean(f'use_leaky_relu') else 'relu'
    input_tensor = layers.Input(shape=[32,  32,  3])
    if hp.Boolean('use_preprocessing'):
        x = layers.RandomRotation(factor=0.1488228133769420)(input_tensor)
        x = layers.RandomTranslation(height_factor=0.069, width_factor=0.069)(x)
        x = layers.RandomFlip(mode='horizontal')(x)
        x = layers.BatchNormalization()(x)
        if hp.Boolean('use_pre_conv'):
            x = layers.Conv2D(filters=hp.Int('conv_filter', min_value=64, max_value=128, step=64),
                          kernel_size=3,
                          activation=activation_function, padding='same',
                          kernel_regularizer=keras.regularizers.l2(0.003) if hp.Boolean('l2_reg') else None)(x)
        x = MBConv1(x, hp=hp)
    else:
        if hp.Boolean('use_pre_conv'):
            x = layers.Conv2D(filters=hp.Int('conv_filter', min_value=64, max_value=128, step=64),
                          kernel_size=3,
                          activation=activation_function, padding='same',
                          kernel_regularizer=keras.regularizers.l2(0.003) if hp.Boolean('l2_reg') else None)(input_tensor)
            x = MBConv1(x, hp=hp)
        else:
            x = MBConv1(input_tensor, hp=hp)

        
    x = MBConv6(x, hp=hp, out_channels=hp.Int('out_channels_1', min_value=20, max_value=44, step=12),
                expansion=hp.Int('expansion_1', min_value=4, max_value=8, step=2),
                use_se=hp.Boolean('use_se_1'),
                dropout_rate=hp.Float('conv6_dropout_rate_1', min_value=0.0, max_value=0.2, step=0.1))
#     if hp.Boolean('use_more_mbconv'):
#         x = MBConv6(x, hp=hp, out_channels=hp.Int('out_channels_1', min_value=20, max_value=44, step=12),
#                 expansion=hp.Int('expansion_1', min_value=4, max_value=8, step=2),
#                 use_se=hp.Boolean('use_se_1'),
#                 dropout_rate=hp.Float('conv6_dropout_rate_1', min_value=0.0, max_value=0.2, step=0.1))
#         x = MBConv6(x, hp=hp, out_channels=hp.Int('out_channels_1', min_value=20, max_value=44, step=12),
#                 expansion=hp.Int('expansion_1', min_value=4, max_value=8, step=2),
#                 use_se=hp.Boolean('use_se_1'),
#                 dropout_rate=hp.Float('conv6_dropout_rate_1', min_value=0.0, max_value=0.2, step=0.1))
    
#     x = MBConv6(x, hp=hp, out_channels=hp.Int('out_channels_2', min_value=20, max_value=44, step=12),
#                 expansion=hp.Int('expansion_2', min_value=4, max_value=8, step=2),
#                 use_se=hp.Boolean('use_se_2'),
#                 dropout_rate=hp.Float('conv6_dropout_rate_2', min_value=0.0, max_value=0.2, step=0.1))
    if hp.Boolean('use_more_mbconv'):
#         x = MBConv6(x, hp=hp, out_channels=hp.Int('out_channels_2', min_value=20, max_value=44, step=12),
#                 expansion=hp.Int('expansion_2', min_value=4, max_value=8, step=2),
#                 use_se=hp.Boolean('use_se_2'),
#                 dropout_rate=hp.Float('conv6_dropout_rate_2', min_value=0.0, max_value=0.2, step=0.1))
        x = MBConv6(x, hp=hp, out_channels=hp.Int('out_channels_2', min_value=20, max_value=44, step=12),
                expansion=hp.Int('expansion_2', min_value=4, max_value=8, step=2),
                use_se=hp.Boolean('use_se_2'),
                dropout_rate=hp.Float('conv6_dropout_rate_2', min_value=0.0, max_value=0.2, step=0.1))
    
    x = layers.Conv2D(filters=hp.Int('conv_filter', min_value=32, max_value=96, step=64),
                      kernel_size=hp.Choice('conv_kernel', values=[3,  7]),
                      activation=activation_function, padding='same',
                      kernel_regularizer=keras.regularizers.l2(0.003) if hp.Boolean('l2_reg') else None)(x)
    x = layers.BatchNormalization()(x)
    if hp.Boolean('use_post_conv'):
        if hp.Boolean('use_maxpool'):
            x = layers.MaxPool2D()(x)
        x = layers.Conv2D(filters=hp.Int('conv_filter', min_value=64, max_value=128, step=64),
                          kernel_size=hp.Choice('conv_kernel', values=[3,  7]),
                          activation=activation_function, padding='same',
                          kernel_regularizer=keras.regularizers.l2(0.003) if hp.Boolean('l2_reg') else None)(x)
        if hp.Boolean('use_maxpool'):
            x = layers.MaxPool2D()(x)
        x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    

    for i in range(hp.Int('num_dense_layers', min_value=4, max_value=10, step=6)):   
        x = layers.Dense(hp.Int(f'dense_units', min_value=256, max_value=512, step=256),
                         activation=activation_function,
                         kernel_regularizer=keras.regularizers.l2(0.003) if hp.Boolean('l2_reg') else None)(x)
        x = layers.Dropout(hp.Float(f'dropout_rate', min_value=0.0, max_value=0.4, step=0.2))(x)
    
    x = layers.Dense(1, activation='sigmoid')(x)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=0.005, max_value=0.08, sampling='LOG'),
                                         epsilon=0.01)
    model = Model(inputs=input_tensor, outputs=x)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy'])
    return model


In [17]:
tuner = Hyperband(
    build_model,
    objective='val_binary_accuracy',
    max_epochs=45,
    factor=3,
    directory='/kaggle/working/projects',
    project_name='cifake_hyperband_6')

In [ ]:
# Start the hyperparameter search with early stopping
tuner.search(ds_train, epochs=45, validation_data=ds_valid, callbacks=[early_stopping, tensorboard, reduce_lr, model_checkpoint])

best_model = tuner.get_best_models(num_models=1)[0]


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
False             |False             |use_leaky_relu
False             |False             |use_preprocessing
True              |True              |use_pre_conv
44                |44                |out_channels_1
4                 |4                 |expansion_1
False             |False             |use_se_1
0.1               |0.1               |conv6_dropout_rate_1
True              |True              |use_more_mbconv
32                |32                |conv_filter
3                 |3                 |conv_kernel
False             |False             |l2_reg
True              |True              |use_post_conv
10                |10                |num_dense_layers
512               |512               |dense_units
0                 |0                 |dropout_rate
0.0051471         |0.0051471         |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tun

In [ ]:
tuner.results_summary()

In [ ]:
best_model.summary()

In [ ]:
best_model.save(f"/kaggle/working/BEST{NAME}.keras")